In [1]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
from dataset import DataPrep
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from mlp import MLP
from sklearn.metrics import mean_squared_error

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
df = pd.read_csv('data/processed_2010-2017.csv')
df = df[['spot_return', 'delta', 'ttm_yr', 'impvol_chg']]
df = torch.tensor(df.values).float().to(device)

In [10]:
dataset = DataPrep(df)
train, test = dataset.split(test_ratio=0.3)
train_dl = DataLoader(train, batch_size=512, shuffle=True)
test_dl = DataLoader(test, batch_size=512, shuffle=False)

In [11]:
def validate(test_dl, model, criterion):
    predictions, actuals = torch.FloatTensor().to(device), torch.FloatTensor().to(device)
    for i, (inputs, targets) in enumerate(test_dl):
        with torch.no_grad():
            yhat = model(inputs)
            predictions = torch.cat((predictions, yhat), 0)
            actuals = torch.cat((actuals, targets), 0)
    actuals = actuals.unsqueeze(dim=1)
    mse = criterion(predictions, actuals).item()
    return mse

In [15]:
model = MLP(input_dim=3).to(device=device)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Training
for epoch in range(10):
    for i, (inputs, targets) in enumerate(train_dl):
        optimizer.zero_grad()
        yhat = model(inputs)
        loss = criterion(yhat, targets.unsqueeze(dim=1))
        loss.backward()
        optimizer.step()
    if epoch % 1 == 0:
        mse = validate(test_dl, model, criterion)
        print("Done with training epoch No." + str(epoch) + ". MSE is " + str(mse))

Done with training epoch No.0. MSE is 0.012228376232087612
Done with training epoch No.1. MSE is 0.012209395878016949
Done with training epoch No.2. MSE is 0.012202967889606953
Done with training epoch No.3. MSE is 0.012199738062918186
Done with training epoch No.4. MSE is 0.01219780184328556
Done with training epoch No.5. MSE is 0.012196516618132591
Done with training epoch No.6. MSE is 0.012195602059364319
Done with training epoch No.7. MSE is 0.012194923125207424
Done with training epoch No.8. MSE is 0.012194397859275341
Done with training epoch No.9. MSE is 0.012193979695439339


In [8]:
eval_model(test_dl, model)

0.012190218083560467